### Modeling a bidirectional reaction

Modeling the bidirectional reaction of oxygen binding to myoglobin will be almost identical to the one-way reaction we already looked at. The derivatives of the system are slightly more complex, as each derivative now consists of two terms, instead of one. However, the underlying principles, and the code needed to solve the ODE system is almost identical.

It is now your turn to try to set up the ODE system, and solve it using Python. Take a look at section two of the accompanying notebook:
* <a href="E2 - Exercises on Physical Chemistry.ipynb#equilibrium">Exercise 1: Modeling a Two-way Reaction to Understand Equilibrium</a>

<a id="enzymes"></a>
## Enzyme Kinetics

Enzymes are proteins that act as catalysts for biochemical reactions, meaning they speed up the reactions, without being consumed in the process. There are many different mechanisms an enzyme can do this, but we won't go into the mechanistic details of it now. Instead, we want to make a simplified and understandable model of how enzymes work.

Enzymes generally work by somehow lowering the energy barrier of a chemical reaction. This can for example be achieved by binding to an intermediate state in the reaction. Recall now Arrhenius' law, which stated that the rate constant of a reaction could be written as
$$k = Ae^{-E_{\rm barrier}/k_{\rm B}T}.$$
This relation tells us that the rate of a reaction is proportional to the exponential of the activation energy. Thus, even a relatively small reduction in the energy barrier of a given reaction can cause a huge change in reaction speed. Because of this, enzymes are usually highly efficient and can speed up reactions by many orders of magnitude ($>10^7$ in some cases!). 

<img src="fig/activation_barrier_enzyme.png" width=400>

The drastic change in reaction speed due to enzymes is hugely important for cellular function. We can effectively have reacting systems in the cell that have a rate so slow that they are effectively stable, but as soon as a bit of enzyme is added, things start happening. In this sense, enzymes can be thought of as on-buttons for various processes in the cell that are lying in standby. An example of this is the energy carrier molecules ATP. These molecules carry a fair bit of chemical energy, and so will spontaneously dissociate and release energy. However, the rate of this dissociation is very slow, and so it takes a very long time for ATP to dissociate on its own. The cell can therefore produce ATP through metabolism and let it sit for a while, without fear of wasting energy. Molecular machines can now be equipped with enzymes that speed up the dissociation of ATP, harnessing some of the released energy in the process. These enzymatic machines are known as *ATPases*, and we will see some of examples of these later in the course.

#### Enzymes change reaction speed, not direction

It is important to note that an enzyme can never change the net change in Gibb's energy in a reaction, and will therefore never be able to change the direction of a reaction. An enzyme cannot trigger a reaction that is not spontaneous on its own, it can just speed it up. The upside however, is that the enzyme molecules doesn't get used up in the reaction, and therefore even a small amount of enzyme is usually enough to trigger some process.

<a id="michaelismenten"></a>
### Modeling enzymatic reaction

The exact action of enzymes can vary a lot, and can be challenging to explore properly due to their specific nature and the complexity of large proteins. For example, the exact way the enzyme *catalase* works remains elusive, despite it being common to more or less all life that processes oxygen. So we won't try to model *how* an enzyme works, but instead what it does.

We will now look at how we can model the activity of an enzyme with a simple model. While simple, this model is incredibly useful, and sees a  lot of use in physiological modeling. As it is a simplified model with some underlying assumptions, it is often referred to as first-order kinetics.

Enzymes work by binding to some molecule, the *substrate*, converting it, and then releasing the *product*. We therefore propose the following reaction to model this activity
$${\rm S} + {\rm E} \underset{k_{-1}}{\overset{k_1}{\rightleftharpoons}} {\rm ES} \overset{k_2}{\rightarrow} {\rm E} + {\rm P}.$$
Here S is the substrate that is to be converted and E is the enzyme, these can bind together and form an enzyme-substrate complex, ES. The ES complex can either break up again without converting the substrate, or the substrate can be converted and the complex breaks up into a free enzyme molecule and a product molecule, P. We assume that once the product has been released by the enzyme, it no longer binds to the enzyme spontaneously, so there is no reverse arrow in the final reaction.

We want to find the rate at which the product is made, i.e., the time derivative $\dot{[{\rm P}]}$. We will call this rate the 'reaction velocity' $v$. Luckily, the law of mass action applies to each concentration separately, so looking at the reaction equation, we can write this out as
$$v = \dot{[{\rm P}]} = k_2[{\rm ES}],$$
as you can see, to compute an actual value, we first need to find an expression for $[ES]$. We start using the law of mass action for $E$:
$$[\dot{{\rm E}}] = (k_{-1} + k_2)[{\rm ES}] - k_1[{\rm S}][{\rm E}].$$
To go further, we will make an assumption that the system will quickly reach an equilibrium where the ratio of bound and unbound enzyme is constant, and so $[{\rm E}]$ will be constant. Such an assumption is sometimes referred to as *quasistatic*. This means we can set the derivative to zero to get
$$(k_{-1} + k_2)[{\rm ES}] = k_1[{\rm S}][{\rm E}].$$
The enzyme doesn't get "used up", so the total amount of enzyme is constant over time, so we can eliminate $[E]$ using the relation $[E] + [{\rm ES}] = [{\rm E}]_{\rm tot}$, giving
$$(k_{-1} + k_2)[{\rm ES}] = k_1[S]([{\rm E}]_{\rm tot} - [{\rm ES}])$$
Solving for $[{\rm ES}]$ gives
$$[{\rm ES}] = \frac{k_1[{\rm S}][{\rm E}]_{\rm tot}}{k_{-1} + k_2 + k_1[{\rm S}]}$$

Going back and inserting this into the reaction velocity gives
$$v = \frac{k_1 k_2 [{\rm S}][{\rm E}]_{\rm tot}}{k_{-1} + k_2 + k_1[{\rm S}]}.$$
This might look very messy, but remember that most of these are just constants, which we can recombine to simplify things. Defining $v_{\rm max} \equiv k_2[{\rm E}]_{\rm tot}$ gives
$$v = v_{\rm max} \frac{k_1 [{\rm S}]}{k_{-1} + k_2 + k_1[{\rm S}]}.$$
Next we divide by $k_1$ in the fraction and define $K_{m} = (k_{-1} + k_2)/k_1$, giving
$$v = v_{\rm max} \frac{[{\rm S}]}{K_m + [{\rm S}]}.$$
Note that $K_m > 0$, so that $[{\rm S}]/(K_m + [{\rm S}])<1$, meaning the reaction velocity can be in the range $[0, v_{\rm max}]$.

If $[S] = K_m$, we get $v = v_{\rm max}/2$, thus $K_m$ can be thought of as the point where the enzyme is half-saturated.

The model we have just derived is known as Michaelis-Menten kinetics. It works very well for a broad range of enzymes. The cases were it might not work that well are cases where we have several different substrates. Single substrate cases are usually assumed to follow Michaelis-Menten kinetics.

Also, take a note to compare the Michaelis-Menten kinetics with the hill equation. Note how similar these are. In fact, the Michaelis-Menten equation is a special case of the Hill equation where $n=1$, meaning we assume there is no cooperativity involved.

### Enzyme Inhibition

Another class of molecules are the enzyme inhibitors. These are molecules that reduce, or completely shut of, the activity of enzymes. There are different functional ways for inhibitors to work, but the simplest mechanism is simply binding to enzyme more readily than the actual substrate, but then not actually converting to anything. Thus the enzymes waste their binding time on the inhibitors rather than the substrate. Enzyme inhibition is another important tool in the cell for regulating behavior, and many drugs are enzyme inhibitors. An example of this is penicillin, which inhibits an enzyme bacteria need for cell wall synthesis, eventually killing the bacteria as it is unable to maintain its own cell wall.

Extending the Michaelis-Menten model to include enzyme inhibitors is quite easy, we simply add the inhibitor as a new species $I$, and new reactions to the system where the inhibitor can bind to the enzyme. It then turns out we can make slightly different models, depending on where the inhibitor binds. These different models would behave differently in practice. When comparing to experiment, it turns out that the different models correspond to different inhibitors, and so in practice, different enzyme inhibitors are grouped into different classes. The figure below shows how the Michaelis-Menten model would be extended for each of these groups. 

<img src="fig/enzyme_inhibition.png" width=800>

For each of these cases, we could go through our analysis again, setting up the kinetics from the law of mass action, assume equilibrium and quasi-static conditions, and find expressions for the first-order enzyme kinetics. While we won't go through all the algebra here, the model predictions we end up with agree well with experiment, and are often very useful in practice.

As you have seen, the Michaelis-Menten model is quite simple, but surprisingly powerful, the fact that it can so easily be extende to include the effects of inhibitors is another reason it is so often used in biomedical modeling.

#### Important Points

* Enzymes are proteins that acts as catalysts for specific biochemical reactions
* We can model first-order enzyme kinematics with the Michaelis-Menten model
* The equation is
$$v = \frac{\rm d [P]}{{\rm d}t} = v_{\rm max} \frac{\rm [S]}{K_{\rm m} + [{\rm S}]},$$
where $v_{\rm max}$ is the maximum reaction velocity, which is proportional to the amount of total enzyme $[E]_{\rm tot}$, and $K_{\rm m}$ is the Michaelis constant. When $[S] = K_{\rm M}$ the enzyme is half-saturated with $v = \frac{1}{2}v_{\rm max}$.
* The Michaelis-Menten model can easily be extended to include enzyme inhibitors

### Solving the Michaelis-Menten system

So far, we have made some assumptions to simplify the mathematics, so we can find some useful analytical expressions for the Micahelis-Menten model. We can however, also solve the ODE-system we derived numerically using the same approach as we just outlined for other chemical reactions. The following exercise deals with this:
* <a href="E2 - Exercises on Physical Chemistry.ipynb#kinetics">Exercise 2: Modeling Michaelis-Menten Kinetics</a>

#### Important Points on the Law of Mass action:
* The law of mass action states that the rate of a reaction is proportional to the concentration of the reactants
* The proportionality factor is the *rate constant*, often written as a lower-case $k$
* It is called the rate *constant* because it does not vary with the concentrations of the reactants, it might vary with other conditions such as temperature
* From the law of mass action we can set up a system of ODEs governing the time evolution of the system

#### Important points on chemical equilibrium
* Most chemical reactions are reversible, and can proceed in either forward or reverse direction
* The forward and reverse directions have separate, and different, rate constants
* At chemical equilibrium, the rates of the two directions balance out, and concentrations do not change
* Equilibrium can be characterized by the dissociation constant, which is the ratio of the two rate constants $K_{\rm d} = k_-/k_-.$

#### Important Points

* The Hill equation describes the binding of some ion or molecule (a *ligand*) to a macromolecule such as a protein
* The Hill equation is
$$\theta = \frac{[L]^n}{K_{\rm d} + [L]^n},$$
where $\theta$ is the fractional binding, $[L]$ the ligand-concentration, $K_{\rm d}$ the dissociation constant of the binding reaction and $n$ the Hill coefficient.
* For many proteins found in the cell, binding is enhanced if there are already other molecules bound to the protein, this is known as *cooperative binding*
* Cooperative binding is captured by the Hill equation and corresponds to $n > 1$. The larger the Hill coefficient, the stronger the cooperativity.